In [1]:
import numpy as np

In [2]:
DEFAULT_THRESH = 10**-12

In [3]:
def f_1(x):
    return np.exp(-(x**2)/2)

def i_simpson(a,b,n,fofx):
    h = (b-a)/n
    i_simp = fofx(a)/6 + fofx(b)/6
    for i in range(1,n):
        i_simp += fofx(a+i*h)/3
    for i in range(1,n+1):
        i_simp += 2*fofx(a+(i-0.5)*h)/3
    return h*i_simp

def norm_cdf(x, n):
    if x > 0:
        return 0.5 + (1 / np.sqrt(2 * np.pi)) * i_simpson(0,x,n,f_1)
    elif x < 0:
        return 0.5 - (1 / np.sqrt(2 * np.pi)) * i_simpson(x,0,n,f_1)
    else:
        return 0.5

def norm_cdf_thresh(x, thresh):
    n_0, n = 4, 8
    i_old, i_new = norm_cdf(x, n_0), norm_cdf(x, n)
    #print("using n = %s, estimated integral = %s" % (n_0, i_old))
    #print("using n = %s, estimated integral = %s" % (n, i_new))
    while(np.abs(i_new - i_old) > thresh):
        i_old = i_new
        n = 2 * n
        i_new = norm_cdf(x, n)
        #print("using n = %s, estimated integral = %s" % (n, i_new))
    return i_new

def norm_cdf_def_thresh(x):
    return norm_cdf_thresh(x, DEFAULT_THRESH)

def norm_cdf_approx(x):
    z = np.abs(x)
    y = 1 / (1 + 0.2316419 * z)
    a1, a2, a3, a4, a5 = 0.319381530, -0.356563782, 1.781477937, -1.821255978, 1.330274429
    m = 1 - np.exp((-x**2)/2)*(a1*y + a2*y**2 + a3*y**3 + a4*y**4 + a5*y**5) / np.sqrt(2*np.pi)
    if x > 0:
        return m
    else:
        return 1-m

In [4]:
def bs_price(T,isCall,S,K,vol,r,q,n_cdf):
    d1 = (np.log(S / K) + (r - q + (vol ** 2)/2) * T) / (vol * np.sqrt(T))
    d2 = d1 - (vol * np.sqrt(T))

    if isCall:
        #call
        return S * np.exp(-q * T) * n_cdf(d1) - K * np.exp(-r * T) * n_cdf(d2)
    else:
        #put
        return K * np.exp(-r * T) * n_cdf(-d2) - S * np.exp(-q * T) * n_cdf(-d1)

In [5]:
def bs_delta(T,isCall,S,K,vol,r,q,n_cdf):
    d1 = (np.log(S / K) + (r - q + (vol ** 2)/2) * T) / (vol * np.sqrt(T))
    if isCall:
        return n_cdf(d1)
    else:
        return -n_cdf(-d1)

In [6]:
# start here
put_pos = 1000
S = 25
K = 30
T = 0.25
vol = 0.3
isCall = False
r = 0.02
q = 0

stock_pos = 400
cash_pos = 10000

In [7]:
put_price = bs_price(T,isCall,S,K,vol,r,q,norm_cdf_def_thresh)
put_delta = bs_delta(T,isCall,S,K,vol,r,q,norm_cdf_def_thresh)

In [8]:
print(put_price, put_delta)

5.08836140106 -0.86588410042


### Part i)

In [9]:
port_val = put_pos * put_price + stock_pos * S + cash_pos
print("Value of the portfolio is:")
print(port_val)

Value of the portfolio is:
25088.3614011


### Part ii)

In [10]:
dollar_delta = put_pos * put_delta * S
dollar_long = stock_pos * S
net_delta = dollar_delta + dollar_long
print("delta of the puts=%s, stock long MV=%s" % (dollar_delta, dollar_long))
print("net delta of the portfolio:",net_delta)

delta of the puts=-21647.1025105, stock long MV=10000
net delta of the portfolio: -11647.1025105


In [11]:
stock_to_buy = -net_delta / S
print("need to buy %s more stock" % stock_to_buy)

need to buy 465.88410042 more stock


In [12]:
stock_pos = 400
stock_pos += stock_to_buy
cash_pos -= stock_to_buy * S
print("stock pos=%s, stock mv=%s, cash_pos=%s" % (stock_pos, stock_pos*S, cash_pos))

stock pos=865.88410042, stock mv=21647.1025105, cash_pos=-1647.10251051


In [13]:
dollar_delta = put_pos * put_delta * S
dollar_long = stock_pos * S
net_delta = dollar_delta + dollar_long
print("hedged delta of the portfolio:", net_delta)

hedged delta of the portfolio: 0.0


### Part iii)

In [14]:
S_list = [30, 26, 22, 27]
cur_net_delta = 0

print("At the Beginning:")
print("Options Pos=%s, Stock Pos=%s, Cash=%s" % (put_pos, stock_pos, cash_pos))

weeks_passed = 1
for S in S_list:
    print('\nCalculating for after %s weeks passed, price of stock=%s' % (weeks_passed, S))
    T = 0.25 - weeks_passed / 52
    print("T=%s" % T)
    
    # calculating the put pricing
    put_price = bs_price(T,isCall,S,K,vol,r,q,norm_cdf_def_thresh)
    put_delta = bs_delta(T,isCall,S,K,vol,r,q,norm_cdf_def_thresh)
    print("put price=%s, put delta=%s" % (put_price, put_delta))
    
    # calculating the cash price
    cash_pos = cash_pos * np.exp(r*(1/52))
    print("cash at this time is:%s" % cash_pos)
    
    port_val = put_pos * put_price + stock_pos * S + cash_pos
    print("Portfolio Value is:", port_val)
    print("Before Hedging positions are:")
    print("Options Pos=%s, Stock Pos=%s, Cash=%s" % (put_pos, stock_pos, cash_pos))
    
    cur_net_delta = put_pos * put_delta * S + stock_pos * S
    print("net delta of the portfolio is now: ", cur_net_delta)
    
    print("old stock pos=%s" % stock_pos)
    stocks_to_trade = -cur_net_delta / S
    stock_pos += stocks_to_trade 
    print("Trade is: %s stock" % stocks_to_trade)
    print("new stock pos=%s" % stock_pos)
    cash_pos += -stocks_to_trade * S
    port_val = put_pos * put_price + stock_pos * S + cash_pos
    print("Portfolio Value is:", port_val)
    print("After Hedging, positions are:")
    print("Options Pos=%s, Stock Pos=%s, Cash=%s" % (put_pos, stock_pos, cash_pos))
    cur_net_delta = put_pos * put_delta * S + stock_pos * S
    print("Portfolio Delta=%s" % (cur_net_delta))
    weeks_passed += 1
    

At the Beginning:
Options Pos=1000, Stock Pos=865.88410042, Cash=-1647.10251051

Calculating for after 1 weeks passed, price of stock=30
T=0.23076923076923078
put price=1.6511607267, put delta=-0.45855162178
cash at this time is:-1647.73613331
Portfolio Value is: 25979.947606
Before Hedging positions are:
Options Pos=1000, Stock Pos=865.88410042, Cash=-1647.73613331
net delta of the portfolio is now:  12219.9743592
old stock pos=865.88410042
Trade is: -407.33247864 stock
new stock pos=458.55162178
Portfolio Value is: 25979.947606
After Hedging, positions are:
Options Pos=1000, Stock Pos=458.55162178, Cash=10572.2382259
Portfolio Delta=1.81898940355e-12

Calculating for after 2 weeks passed, price of stock=26
T=0.21153846153846154
put price=4.1893333374, put delta=-0.825739334597
cash at this time is:10576.3052534
Portfolio Value is: 26687.9807571
Before Hedging positions are:
Options Pos=1000, Stock Pos=458.55162178, Cash=10576.3052534
net delta of the portfolio is now:  -9546.88053324